In [5]:
import numpy as np 

import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json

from geopy.geocoders import Nominatim

import requests

from bs4 import BeautifulSoup

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print("Libraries imported.")

Libraries imported.


In [6]:
# create three lists to store table data
postalCodeList=[]
boroughList=[]
neighborhoodList=[]

In [7]:
data= requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup= BeautifulSoup(data, 'html.parser')

In [8]:
# find the table
soup.find('table').find_all('tr')

# find all the rows of table
soup.find('table').find_all('tr')

#for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells= row.find_all('td')

In [9]:
#append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells= row.find_all('td')
    if(len(cells)>0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [18]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList, 
                           "Borough":boroughList, 
                           "Neighborhood":neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [19]:
toronto_df_dropna= toronto_df[toronto_df.Borough !="Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [20]:
#group neighborhoods in the same borough
toronto_df_grouped= toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ",".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [21]:
# For Neighborhood="Not Assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"]== "Not Assigned":
        row["Neighborhood"]= row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [22]:
#create new test frame
column_names=["PostalCode", "Borough", "Neighborhood"] 
test_df= pd.DataFrame(columns=column_names)

test_list= ["M5G","M2H","M4B", "M1J", "M4G", "M4M","M1R","M9V","M9L","M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df= test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)

test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Woodbine Gardens,Parkview Hill"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Maryvale,Wexford"
7,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf..."


In [35]:
#print the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(103, 3)

8. Load the coordinates from the CSV file on Coursera

In [24]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
#rename the coloumn PostalCode
coordinates.rename(columns={"PostalCode": "PostalCode"}, inplace=True)
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge two tables to get the coordinates.

In [41]:
#merge two table on the column"PostalCode"

toronto_df_new = toronto_df_grouped.merge(coordinates, on ="PostalCode")

toronto_df_new.head()

KeyError: 'PostalCode'

In [42]:
#create new test frame
column_names=["PostalCode", "Borough", "Neighborhood","Longitude", "Latitude"] 
test_df= pd.DataFrame(columns=column_names)

test_list= ["M5G","M2H","M4B", "M1J", "M4G", "M4M","M1R","M9V","M9L","M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df= test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)

test_df

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,Borough,Latitude,Longitude,Neighborhood,PostalCode
0,Downtown Toronto,NaN,NaN,Central Bay Street,M5G
1,North York,NaN,NaN,Hillcrest Village,M2H
2,East York,NaN,NaN,"Woodbine Gardens,Parkview Hill",M4B
3,Scarborough,NaN,NaN,Scarborough Village,M1J
4,East York,NaN,NaN,Leaside,M4G
5,East Toronto,NaN,NaN,Studio District,M4M
6,Scarborough,NaN,NaN,"Maryvale,Wexford",M1R
7,Etobicoke,NaN,NaN,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",M9V
8,North York,NaN,NaN,Humber Summit,M9L
9,Downtown Toronto,NaN,NaN,"CN Tower,Bathurst Quay,Island airport,Harbourf...",M5V
